# Divisão de um município em setores censitários e bairros

Itens a serem vistos:

- como baixar as malhas de outros estados do Brasil
- como baixar o arquivo com as variáveis divididas por setores censitários de MG
- plotar bairros de Varginha (divisão por setores)
- plotar nomes dos bairros de outros municípios de MG (Passos e BH)
- arquivo 'basico_MG.csv' do IBGE
- unir as informações geográficas dos setores de VGN com o 'basico_MG.csv'

### Setores censitários

Para baixar as malhas dos outros estados do Brasil ir em:

https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/censo_2010/setores_censitarios_shp/

Para baixar o arquivo com as variáveis por setores censitários em MG:

https://ftp.ibge.gov.br/Censos/Censo_Demografico_2010/Resultados_do_Universo/Agregados_por_Setores_Censitarios/

In [ ]:
!pip install geopandas shapely mapclassify libpysal splot esda pysal
!pip install --upgrade pyshp
!pip install --upgrade descartes

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as sm

# para gráficos
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# para a análise de dados espaciais
import geopandas as gp
import pysal as ps
import splot
import mapclassify as mc
from libpysal.weights import Queen
from libpysal import weights
from esda import Moran, Moran_Local, G_Local
from splot.esda import plot_moran, moran_scatterplot, lisa_cluster, plot_local_autocorrelation

# links com arquivos úteis
link = 'https://raw.githubusercontent.com/lincolnfrias/dados/master/'
link_p = 'https://raw.githubusercontent.com/patriciasiqueira/patriciasiqueira.github.io/master/arquivos/'

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# estado de Minas Gerais dividido em setores censitários
setores = gp.read_file('/content/drive/My Drive/unifal/analise-espacial/dados/mg_setores_censitarios/31SEE250GC_SIR.shp')

In [ ]:
# estado de Minas Gerais dividido em setores censitários
# setores = gp.read_file('31SEE250GC_SIR.shp')

In [ ]:
setores.head()

,ID,CD_GEOCODI,TIPO,CD_GEOCODB,NM_BAIRRO,CD_GEOCODS,NM_SUBDIST,CD_GEOCODD,NM_DISTRIT,CD_GEOCODM,NM_MUNICIP,NM_MICRO,NM_MESO,geometry
0,1,310010405000011,RURAL,None,None,31001040500,None,310010405,ABADIA DOS DOURADOS,3100104,ABADIA DOS DOURADOS,PATROCÍNIO,TRIÂNGULO MINEIRO/ALTO PARANAÍBA,"POLYGON ((-47.57649 -18.22600, -47.57596 -18.2..."
1,2,310010405000012,RURAL,None,None,31001040500,None,310010405,ABADIA DOS DOURADOS,3100104,ABADIA DOS DOURADOS,PATROCÍNIO,TRIÂNGULO MINEIRO/ALTO PARANAÍBA,"POLYGON ((-47.42967 -18.16543, -47.42943 -18.1..."
2,3,310010405000013,RURAL,None,None,31001040500,None,310010405,ABADIA DOS DOURADOS,3100104,ABADIA DOS DOURADOS,PATROCÍNIO,TRIÂNGULO MINEIRO/ALTO PARANAÍBA,"POLYGON ((-47.42649 -18.29056, -47.42643 -18.2..."
3,4,310020305000001,URBANO,310020305013,Zona Central,31002030500,None,310020305,ABAETÉ,3100203,ABAETÉ,TRÊS MARIAS,CENTRAL MINEIRA,"POLYGON ((-45.44671 -19.15200, -45.44651 -19.1..."
4,5,310020305000002,URBANO,310020305006,Fátima,31002030500,None,310020305,ABAETÉ,3100203,ABAETÉ,TRÊS MARIAS,CENTRAL MINEIRA,"POLYGON ((-45.44291 -19.15714, -45.44275 -19.1..."


In [ ]:
setores.NM_MESO.unique()

array(['TRIÂNGULO MINEIRO/ALTO PARANAÍBA', 'CENTRAL MINEIRA',
       'ZONA DA MATA', 'VALE DO RIO DOCE', 'OESTE DE MINAS',
       'VALE DO MUCURI', 'NORTE DE MINAS', 'SUL/SUDOESTE DE MINAS',
       'CAMPO DAS VERTENTES', 'JEQUITINHONHA',
       'METROPOLITANA DE BELO HORIZONTE', 'NOROESTE DE MINAS'],
      dtype=object)

In [ ]:
setores.shape

In [ ]:
# evitar problemas com nomes
setores = setores.to_crs(epsg=3395)

In [ ]:
# a partir do shapefile de setores de MG, selecionar apenas sul de minas
sul = setores.query('NM_MESO == "SUL/SUDOESTE DE MINAS"')
sul.shape

In [ ]:
sul.shape

In [ ]:
setores.to_file('setores-mg.json', driver='GeoJSON')

In [ ]:
sul.to_file('setores-sul-mg.json', driver='GeoJSON')

In [ ]:
# a partir do shapefile de setores de MG, selecionar o município de Varginha e apenas os setores urbanos
vga = setores.query('NM_MUNICIP == "VARGINHA"')
mapa_vga = vga.query('TIPO == "URBANO"')
mapa_vga.shape

In [ ]:
mapa_vga.columns

In [ ]:
# plotar os bairros, porém, a divisão está em setores censitários - as 161 linhas
# (há bairros repetidos)
mapa = mapa_vga
ax = mapa.plot(figsize=(15,15))

for i in mapa.index:
    plt.text(mapa.geometry.centroid[i].coords[0][0], mapa.geometry.centroid[i].coords[0][1], mapa.NM_BAIRRO[i],
             fontsize=10, horizontalalignment='center', verticalalignment='center')

ax.set_axis_off();

In [ ]:
# tentar agrupar em bairros
# mapa contém as informações divididas em 161 setores censitários e queremos unir os setores em 38 bairros
# uso da função dissolve
# http://geopandas.org/aggregation_with_dissolve.html
bairros = mapa_vga.dissolve(by='NM_BAIRRO')
bairros.head()

In [ ]:
# plotar os bairros usando o novo dataframe bairros
# Obs.: os índices ficaram como os nomes dos bairros, por isso, criei uma nova coluna NM_BAIRRO para
# colocar os nomes dos bairros e a nova coluna index irá de 0 a 37 (número de bairros)
bairros['NM_BAIRRO'] = bairros.index
bairros.index = range(len(bairros))
bairros.head()

In [ ]:
# plotar os bairros
ax = bairros.plot(figsize=(15,15), cmap='Accent')

for i in bairros.index:
    plt.text(bairros.geometry.centroid[i].coords[0][0], bairros.geometry.centroid[i].coords[0][1], bairros.NM_BAIRRO[i],
             fontsize=10, horizontalalignment='center', verticalalignment='center')

ax.set_axis_off();

## Plotar os nomes dos bairros de outro município de MG:

In [ ]:
# para saber quais municípios possuem os nomes dos bairros disponíveis
setores.NM_BAIRRO.fillna('nnnn', inplace=True)  # preencher os que não têm nomes dos bairros com a string 'nnnn'
setores = setores.query('NM_BAIRRO != "nnnn"')

In [ ]:
# retornar esses municípios
setores['NM_MUNICIP'].unique()

In [ ]:
# plotar o mapa sem os nomes
pss = setores.query('NM_MUNICIP == "PASSOS"')
mapa_pss = pss.query('TIPO == "URBANO"')
mapa_pss.plot(figsize=(9,9)).set_axis_off();

In [ ]:
# verificar se existem bairros para o município escolhido
# dataframe.NM_BAIRRO
pss.NM_BAIRRO

In [ ]:
mapa_pss.columns

In [ ]:
# plotar os bairros, porém, a divisão está em setores censitários
# (há bairros repetidos)
mapa = mapa_pss
ax = mapa.plot(figsize=(15,15))

for i in mapa.index:
    plt.text(mapa.geometry.centroid[i].coords[0][0], mapa.geometry.centroid[i].coords[0][1], mapa.NM_BAIRRO[i],
             fontsize=10, horizontalalignment='center', verticalalignment='center')

ax.set_axis_off();

In [ ]:
# tentar agrupar em bairros
bairros = mapa_pss.dissolve(by='NM_BAIRRO')
bairros.head()

In [ ]:
# plotar os bairros usando o novo dataframe bairros
# Obs.: os índices ficaram como os nomes dos bairros, por isso, criei uma nova coluna NM_BAIRRO para
# colocar os nomes dos bairros e a nova coluna index irá de 0 a 37 (número de bairros)
bairros['NM_BAIRRO'] = bairros.index
bairros.index = range(len(bairros))
bairros.head()

In [ ]:
# plotar os bairros
ax = bairros.plot(figsize=(15,15), cmap='Accent')

for i in bairros.index:
    plt.text(bairros.geometry.centroid[i].coords[0][0], bairros.geometry.centroid[i].coords[0][1], bairros.NM_BAIRRO[i],
             fontsize=10, horizontalalignment='center', verticalalignment='center')

ax.set_axis_off();

**Tarefa:**

Escolher outro município que possua informações sobre os bairros no sul de MG e plotar os bairros, como eu fiz para Varginha e Passos.


### Belo Horizonte

In [ ]:
# plotar o mapa sem os nomes
bh = setores.query('NM_MUNICIP == "BELO HORIZONTE"')
mapa_bh = bh.query('TIPO == "URBANO"')
mapa_bh.plot(figsize=(15,15)).set_axis_off();

In [ ]:
bh.NM_BAIRRO

In [ ]:
mapa_bh.columns

In [ ]:
mapa = mapa_bh

In [ ]:
# tentar agrupar em bairros
bairros = mapa_bh.dissolve(by='NM_BAIRRO')
bairros.head()

In [ ]:
# plotar os bairros usando o novo dataframe bairros
# Obs.: os índices ficaram como os nomes dos bairros, por isso, criei uma nova coluna NM_BAIRRO para
# colocar os nomes dos bairros e a nova coluna index irá de 0 a 37 (número de bairros)
bairros['NM_BAIRRO'] = bairros.index
bairros.index = range(len(bairros))
bairros.head()

In [ ]:
# plotar os bairros
ax = bairros.plot(figsize=(30,30), cmap='Accent')

for i in bairros.index:
    plt.text(bairros.geometry.centroid[i].coords[0][0], bairros.geometry.centroid[i].coords[0][1], bairros.NM_BAIRRO[i],
             fontsize=10, horizontalalignment='center', verticalalignment='center', fontdict={'color':'white'})

ax.set_axis_off();

### Agora vamos pegar variáveis sobre os setores censitários de MG e utilizar os de Varginha.

O arquivo que usaremos será o 'basico_MG.csv'

As variáveis desse arquivo estão codificadas, para saber o que significam, consultar o arquivo 'base_setores_censitarios_censo_2010.pdf'

In [ ]:
# carregar arquivo com dados sobre os setores censitários de MG
mg = pd.read_csv('Basico_MG.csv', encoding='latin1', sep=';', decimal=',')
mg.head()

In [ ]:
mg.columns

In [ ]:
mg.Nome_do_municipio.unique()

In [ ]:
# informações dos setores censitários de Varginha
mg.query('Nome_do_municipio == "VARGINHA"')

In [ ]:
# salvar as informações de Varginha num dataframe
vga_df = mg.query('Nome_do_municipio == "VARGINHA"')
vga_df = vga_df.query('Situacao_setor == [1, 2, 3]')   # só setores urbanos
vga_df.head()

In [ ]:
vga_df.shape    # 161 setores censitários

In [ ]:
vga_df.columns   # variáveis: 12

In [ ]:
# o objeto 'mapa_vga' contém as informações geográficas de todos os setores censitários de Varginha
# vamos incluir as informações das variáveis presentes em 'vgn_df' neste dataframe
# e chamá-lo de geodf
# como usaremos a coluna de código do setor censitário como a variável para unir os dois conjuntos de
# dados, o nome dessa coluna deve ser o mesmo nos dois (chamarei de 'Cod_setor')
df = vga_df
geodf = mapa_vga
geodf.geometry = geodf.geometry.simplify(0.001)
geodf.rename(columns={'CD_GEOCODI': 'CD_setor'}, inplace=True)
geodf.head()

In [ ]:
geodf.shape

In [ ]:
# os códigos não estão sendo lidos como inteiros, transformá-los em inteiros
geodf.CD_setor = geodf.CD_setor.astype('int64')

In [ ]:
# para que os dois dataframes tenham a coluna de códigos com o mesmo nome,
# alterar para CD_setor
df.rename(columns={'Cod_setor': 'CD_setor'}, inplace=True)

In [ ]:
# unir o dataframe de informações geográficas com o outro dataframe com as variáveis
# Obs.: 'suffixes' é para o caso de haver variáveis de mesmo nome, incluir alguma coisa no final do nome
# em cada dataframe
varginha = pd.merge(geodf, df, on='CD_setor')
varginha

In [ ]:
varginha.columns

In [ ]:
varginha.shape